In [ ]:
# A requirements.txt file is included for installing the complete environment on a local machine.
# However, it is recommended to run this notebook via google colab. This will get the
# environment running much faster and grants free access to a GPU. If running via
# colab skip this installation and install only the packages in the cell below

# install the environment from the requirements.txt file
!cat requirements.txt | xargs -n 1 pip install

     |████████████████████████████████| 1.1 MB 4.1 MB/s 
     |████████████████████████████████| 160 kB 54.1 MB/s 
     |████████████████████████████████| 192 kB 69.5 MB/s 
     |████████████████████████████████| 271 kB 56.8 MB/s 
     |████████████████████████████████| 631 kB 4.2 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654020 sha256=80bc78d237e54dbc3c362c48457dbdfa6548359e7b1dd64b575ba380dc0a23bb
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
     |████████████████████████████████| 111 kB 4.2 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=88fa9bfe21dcf710323a1f9bc59352391aabae6c4511138dc3a6c7c74fbde757
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b

In [ ]:
!pip install -r /content/requirements.txt

In [ ]:
# these packages alone should work if using google colab
!pip install torch torchvision==0.11.1
!pip install transformers==4.13.0
!pip install seqeval==1.2.2
!pip install tensorboardx==2.7.0
!pip install simpletransformers==0.63.3

In [2]:
# Imports
import pandas as pd
import numpy as np
import gc
import requests
import os
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import ClassificationArgs
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score, precision_recall_curve, auc
import torch
import logging

# Verify that GPU is being used
print("Cuda available" if torch.cuda.is_available() is True else "CPU")
print("PyTorch version: ", torch.__version__)

Cuda available
PyTorch version:  1.10.0+cu111


In [3]:
# Mount drive for training data if using google colab, or skip and upload
# training data directly to session storage

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Model Training

In [ ]:
# args
model_args = ClassificationArgs()
model_args.evaluate_during_training = True
model_args.evaluate_during_training_silent = False
model_args.evaluate_during_training_steps = 10
model_args.learning_rate = 1.3276125338114695e-05
model_args.manual_seed = 907
model_args.max_seq_length = 256
model_args.no_cache = False #True
model_args.no_save = False #True
model_args.num_train_epochs = 5
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.train_custom_parameters_only = False
model_args.save_steps = 40

# data import
url = '/content/drive/MyDrive/Data/training_tweets.csv' # file path to training tweets
df = pd.read_csv(url, error_bad_lines=False)

# train test split
train_df = df.sample(frac = 0.8, random_state = 907)
test_df = df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop = True)

# convert strings to lower
train_df['text'] = train_df['text'].str.lower()
test_df['text'] = test_df['text'].str.lower()

# Train the model
model = ClassificationModel(
        "electra",
        "google/electra-base-discriminator",
        use_cuda=True,
        args=model_args
    )

model.train_model(train_df, eval_df = test_df)

In [7]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/397 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

In [12]:
result

{'auprc': 0.8309032736873481,
 'auroc': 0.9012078687684224,
 'eval_loss': 0.39487834930419924,
 'fn': 38,
 'fp': 18,
 'mcc': 0.6276786265993519,
 'tn': 271,
 'tp': 70}

In [ ]:
!zip -r ./outputs/best.zip ./outputs/best_model

  adding: outputs/best_model/ (stored 0%)
  adding: outputs/best_model/pytorch_model.bin (deflated 7%)
  adding: outputs/best_model/vocab.txt (deflated 53%)
  adding: outputs/best_model/optimizer.pt (deflated 23%)
  adding: outputs/best_model/eval_results.txt (deflated 26%)
  adding: outputs/best_model/scheduler.pt (deflated 49%)
  adding: outputs/best_model/special_tokens_map.json (deflated 40%)
  adding: outputs/best_model/tokenizer_config.json (deflated 39%)
  adding: outputs/best_model/training_args.bin (deflated 49%)
  adding: outputs/best_model/config.json (deflated 51%)
  adding: outputs/best_model/model_args.json (deflated 62%)
  adding: outputs/best_model/tokenizer.json (deflated 59%)
